# CUDA Core Tutorial - Low-Level GPU Programming
## Table of Contents

1. Introduction to cuda.core
2. Setting Up Your Environment
3. Understanding CUDA Concepts
4. Memory Management
5. Kernel Compilation and Execution
6. Streams and Synchronization
7. Error Handling
8. Performance Optimization
9. Practical Examples
10. Lab

## 1. Introduction to cuda.core
The `cuda.core` module provides direct access to the CUDA driver API, giving you maximum control over GPU programming. Unlike high-level APIs, cuda.core requires you to manage everything manually:

* Context management: Creating and managing execution contexts
* Memory allocation: Explicitly allocating and freeing GPU memory
* Kernel compilation: Compiling CUDA C/C++ code at runtime
* Synchronization: Managing streams and events

When to use cuda.core:

* Maximum performance control
* Custom memory management patterns
* Integration with existing CUDA C/C++ code
* Fine-grained GPU resource management

## 2. Setting Up Your Environment
### Prerequisites

* NVIDIA GPU with CUDA capability
* CUDA driver version 12.2 or higher
* Python 3.8+

### Installation

In [ ]:
pip install cuda-python numpy

SyntaxError: invalid syntax (2111655890.py, line 1)

### Verification

In [ ]:
from cuda import core
import numpy as np

# Check if CUDA is available
try:
    core.cuInit(0)
    print("CUDA initialized successfully!")
    
    # Get device count
    device_count = core.cuDeviceGetCount()
    print(f"Number of CUDA devices: {device_count}")
    
    # Get device properties
    device = core.cuDeviceGet(0)
    name = core.cuDeviceGetName(device)
    print(f"Device 0: {name}")
    
except Exception as e:
    print(f"CUDA initialization failed: {e}")

## 3. Understanding CUDA Concepts
### Key Terminology
**Host vs Device:**

* **Host**: Your CPU and system memory
* **Device**: Your GPU and video memory

**Execution Model**:

* **Kernel**: A function that runs on the GPU
* **Thread**: Individual execution unit
* **Block**: Group of threads that can cooperate
* **Grid**: Collection of blocks

**Memory Hierarchy**:

* **Global Memory**: Main GPU memory (slow but large)
* **Shared Memory**: Fast memory shared within a block
* **Registers**: Fastest memory, private to each thread

### Creating a CUDA Context

In [ ]:
from cuda import core

# Initialize CUDA
core.cuInit(0)

# Get the first GPU device
device = core.cuDeviceGet(0)

# Create a context for this device
context = core.cuCtxCreate(0, device)

print("CUDA context created successfully!")

# Always clean up when done
# core.cuCtxDestroy(context)

## 4. Memory Management
### Basic Memory Operations

In [ ]:
import numpy as np
from cuda import core

# Initialize CUDA context (assume already done)
core.cuInit(0)
device = core.cuDeviceGet(0)
context = core.cuCtxCreate(0, device)

# Create host data
host_array = np.arange(1000, dtype=np.float32)
print(f"Host array size: {host_array.nbytes} bytes")

# Allocate device memory
device_ptr = core.cuMemAlloc(host_array.nbytes)
print(f"Device memory allocated at: {device_ptr}")

# Copy data from host to device
core.cuMemcpyHtoD(device_ptr, host_array)
print("Data copied to device")

# Allocate memory for result
result_ptr = core.cuMemAlloc(host_array.nbytes)

# Copy data back from device to host
result_array = np.zeros_like(host_array)
core.cuMemcpyDtoH(result_array, device_ptr)
print("Data copied back to host")

# Free device memory
core.cuMemFree(device_ptr)
core.cuMemFree(result_ptr)
print("Device memory freed")

### Memory Transfer Patterns

In [ ]:
def demonstrate_memory_patterns():
    # Pattern 1: Host to Device
    host_data = np.random.random(1000).astype(np.float32)
    device_data = core.cuMemAlloc(host_data.nbytes)
    core.cuMemcpyHtoD(device_data, host_data)
    
    # Pattern 2: Device to Device
    device_copy = core.cuMemAlloc(host_data.nbytes)
    core.cuMemcpyDtoD(device_copy, device_data, host_data.nbytes)
    
    # Pattern 3: Device to Host
    result = np.zeros_like(host_data)
    core.cuMemcpyDtoH(result, device_copy)
    
    # Cleanup
    core.cuMemFree(device_data)
    core.cuMemFree(device_copy)
    
    return result

# Usage
result = demonstrate_memory_patterns()
print(f"Memory transfer completed. Result shape: {result.shape}")

## 5. Kernel Compilation and Execution
### Writing Your First Kernel

In [ ]:
from cuda.core.experimental import Program
import numpy as np

# CUDA C source code
vector_add_source = """
extern "C" __global__ void vector_add(float *a, float *b, float *c, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {
        c[i] = a[i] + b[i];
    }
}
"""

# Compile the kernel
program = Program(source=vector_add_source, name="vector_add")
compiled_program = program.compile()
kernel = compiled_program.get_function("vector_add")

print("Kernel compiled successfully!")

### Executing the Kernel

In [ ]:
def execute_vector_add():
    # Prepare data
    N = 1000
    a = np.arange(N, dtype=np.float32)
    b = np.arange(N, dtype=np.float32)
    c = np.zeros(N, dtype=np.float32)
    
    # Allocate device memory
    d_a = core.cuMemAlloc(a.nbytes)
    d_b = core.cuMemAlloc(b.nbytes)
    d_c = core.cuMemAlloc(c.nbytes)
    
    # Copy data to device
    core.cuMemcpyHtoD(d_a, a)
    core.cuMemcpyHtoD(d_b, b)
    
    # Launch kernel
    block_size = 256
    grid_size = (N + block_size - 1) // block_size
    
    kernel.launch(
        grid=(grid_size,),
        block=(block_size,),
        args=(d_a, d_b, d_c, np.int32(N))
    )
    
    # Copy result back
    core.cuMemcpyDtoH(c, d_c)
    
    # Cleanup
    core.cuMemFree(d_a)
    core.cuMemFree(d_b)
    core.cuMemFree(d_c)
    
    return c

# Execute and verify
result = execute_vector_add()
expected = np.arange(1000, dtype=np.float32) * 2
print(f"Kernel execution successful: {np.allclose(result, expected)}")

### Advanced Kernel Example

In [ ]:
# Matrix multiplication kernel
matmul_source = """
extern "C" __global__ void matrix_multiply(float *A, float *B, float *C, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (row < N && col < N) {
        float sum = 0.0f;
        for (int k = 0; k < N; k++) {
            sum += A[row * N + k] * B[k * N + col];
        }
        C[row * N + col] = sum;
    }
}
"""

def matrix_multiply_gpu(A, B):
    N = A.shape[0]
    assert A.shape == (N, N) and B.shape == (N, N)
    
    # Compile kernel
    program = Program(source=matmul_source, name="matrix_multiply")
    kernel = program.compile().get_function("matrix_multiply")
    
    # Allocate device memory
    d_A = core.cuMemAlloc(A.nbytes)
    d_B = core.cuMemAlloc(B.nbytes)
    d_C = core.cuMemAlloc(A.nbytes)
    
    # Copy data
    core.cuMemcpyHtoD(d_A, A)
    core.cuMemcpyHtoD(d_B, B)
    
    # Launch kernel
    block_size = 16
    grid_size = (N + block_size - 1) // block_size
    
    kernel.launch(
        grid=(grid_size, grid_size),
        block=(block_size, block_size),
        args=(d_A, d_B, d_C, np.int32(N))
    )
    
    # Get result
    C = np.zeros_like(A)
    core.cuMemcpyDtoH(C, d_C)
    
    # Cleanup
    core.cuMemFree(d_A)
    core.cuMemFree(d_B)
    core.cuMemFree(d_C)
    
    return C

# Test matrix multiplication
A = np.random.random((64, 64)).astype(np.float32)
B = np.random.random((64, 64)).astype(np.float32)
C_gpu = matrix_multiply_gpu(A, B)
C_cpu = np.dot(A, B)

print(f"Matrix multiplication correct: {np.allclose(C_gpu, C_cpu, atol=1e-5)}")

## 6. Streams and Synchronization
### Understanding Streams
Streams allow asynchronous execution of CUDA operations. This enables overlapping computation with memory transfers for better performance.

In [ ]:
def demonstrate_streams():
    # Create stream
    stream = core.cuStreamCreate()
    
    # Prepare data
    N = 1000
    a = np.arange(N, dtype=np.float32)
    b = np.arange(N, dtype=np.float32)
    c = np.zeros(N, dtype=np.float32)
    
    # Allocate device memory
    d_a = core.cuMemAlloc(a.nbytes)
    d_b = core.cuMemAlloc(b.nbytes)
    d_c = core.cuMemAlloc(c.nbytes)
    
    # Asynchronous memory copy
    core.cuMemcpyHtoDAsync(d_a, a, stream)
    core.cuMemcpyHtoDAsync(d_b, b, stream)
    
    # Launch kernel in stream
    kernel.launch(
        grid=(4,),
        block=(256,),
        args=(d_a, d_b, d_c, np.int32(N)),
        stream=stream
    )
    
    # Asynchronous copy back
    core.cuMemcpyDtoHAsync(c, d_c, stream)
    
    # Wait for stream to complete
    core.cuStreamSynchronize(stream)
    
    # Cleanup
    core.cuStreamDestroy(stream)
    core.cuMemFree(d_a)
    core.cuMemFree(d_b)
    core.cuMemFree(d_c)
    
    return c

result = demonstrate_streams()
print("Stream execution completed")

### Events for Timing

In [ ]:
def time_kernel_execution():
    # Create events
    start_event = core.cuEventCreate()
    end_event = core.cuEventCreate()
    
    # Prepare data
    N = 1000000
    a = np.random.random(N).astype(np.float32)
    b = np.random.random(N).astype(np.float32)
    
    d_a = core.cuMemAlloc(a.nbytes)
    d_b = core.cuMemAlloc(b.nbytes)
    d_c = core.cuMemAlloc(a.nbytes)
    
    core.cuMemcpyHtoD(d_a, a)
    core.cuMemcpyHtoD(d_b, b)
    
    # Record start time
    core.cuEventRecord(start_event)
    
    # Launch kernel
    block_size = 256
    grid_size = (N + block_size - 1) // block_size
    kernel.launch(
        grid=(grid_size,),
        block=(block_size,),
        args=(d_a, d_b, d_c, np.int32(N))
    )
    
    # Record end time
    core.cuEventRecord(end_event)
    core.cuEventSynchronize(end_event)
    
    # Calculate elapsed time
    elapsed_time = core.cuEventElapsedTime(start_event, end_event)
    
    # Cleanup
    core.cuEventDestroy(start_event)
    core.cuEventDestroy(end_event)
    core.cuMemFree(d_a)
    core.cuMemFree(d_b)
    core.cuMemFree(d_c)
    
    return elapsed_time

execution_time = time_kernel_execution()
print(f"Kernel execution time: {execution_time:.2f} ms")

## 7. Error Handling
### Proper Error Checking

In [ ]:
def safe_cuda_operation():
    try:
        # Initialize CUDA
        core.cuInit(0)
        
        # Get device
        device = core.cuDeviceGet(0)
        
        # Create context
        context = core.cuCtxCreate(0, device)
        
        # Your CUDA operations here
        print("CUDA operations completed successfully")
        
    except Exception as e:
        print(f"CUDA error: {e}")
        
    finally:
        # Always cleanup
        try:
            core.cuCtxDestroy(context)
        except:
            pass

safe_cuda_operation()

### Common Error Patterns

In [ ]:
def handle_common_errors():
    errors = []
    
    # Error 1: Invalid device
    try:
        invalid_device = core.cuDeviceGet(999)
    except Exception as e:
        errors.append(f"Invalid device: {e}")
    
    # Error 2: Memory allocation failure
    try:
        # Try to allocate too much memory
        huge_alloc = core.cuMemAlloc(2**40)  # 1TB
    except Exception as e:
        errors.append(f"Memory allocation failed: {e}")
    
    # Error 3: Invalid kernel launch
    try:
        # Invalid grid/block dimensions
        kernel.launch(grid=(0,), block=(0,), args=())
    except Exception as e:
        errors.append(f"Invalid kernel launch: {e}")
    
    return errors

errors = handle_common_errors()
for error in errors:
    print(error)

## 8. Performance Optimization
### Memory Coalescing

In [ ]:
# Good: Coalesced memory access
coalesced_kernel = """
extern "C" __global__ void coalesced_access(float *data, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {
        data[i] = data[i] * 2.0f;  // Sequential access
    }
}
"""

# Bad: Strided memory access
strided_kernel = """
extern "C" __global__ void strided_access(float *data, int n, int stride) {
    int i = (blockIdx.x * blockDim.x + threadIdx.x) * stride;
    if (i < n) {
        data[i] = data[i] * 2.0f;  // Strided access
    }
}
"""

### Shared Memory Usage

In [ ]:
shared_memory_kernel = """
extern "C" __global__ void shared_memory_example(float *input, float *output, int n) {
    extern __shared__ float shared_data[];
    
    int tid = threadIdx.x;
    int i = blockIdx.x * blockDim.x + tid;
    
    // Load data into shared memory
    if (i < n) {
        shared_data[tid] = input[i];
    }
    __syncthreads();
    
    // Process data in shared memory
    if (tid > 0 && tid < blockDim.x - 1 && i < n) {
        output[i] = (shared_data[tid-1] + shared_data[tid] + shared_data[tid+1]) / 3.0f;
    }
}
"""

def use_shared_memory():
    N = 1000
    data = np.random.random(N).astype(np.float32)
    result = np.zeros_like(data)
    
    # Compile kernel
    program = Program(source=shared_memory_kernel, name="shared_memory_example")
    kernel = program.compile().get_function("shared_memory_example")
    
    # Allocate memory
    d_input = core.cuMemAlloc(data.nbytes)
    d_output = core.cuMemAlloc(result.nbytes)
    
    core.cuMemcpyHtoD(d_input, data)
    
    # Launch with shared memory
    block_size = 256
    shared_mem_size = block_size * 4  # 4 bytes per float
    
    kernel.launch(
        grid=((N + block_size - 1) // block_size,),
        block=(block_size,),
        args=(d_input, d_output, np.int32(N)),
        shared_mem_bytes=shared_mem_size
    )
    
    core.cuMemcpyDtoH(result, d_output)
    
    # Cleanup
    core.cuMemFree(d_input)
    core.cuMemFree(d_output)
    
    return result

smoothed_data = use_shared_memory()
print(f"Shared memory kernel executed successfully")

## 9. Practical Examples {tag here}
### Example 1: Image Convolution

In [ ]:
convolution_kernel = """
extern "C" __global__ void convolution_2d(float *input, float *output, float *kernel, 
                                         int width, int height, int kernel_size) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    
    if (col < width && row < height) {
        float sum = 0.0f;
        int half_kernel = kernel_size / 2;
        
        for (int i = -half_kernel; i <= half_kernel; i++) {
            for (int j = -half_kernel; j <= half_kernel; j++) {
                int input_row = row + i;
                int input_col = col + j;
                
                if (input_row >= 0 && input_row < height && 
                    input_col >= 0 && input_col < width) {
                    int input_idx = input_row * width + input_col;
                    int kernel_idx = (i + half_kernel) * kernel_size + (j + half_kernel);
                    sum += input[input_idx] * kernel[kernel_idx];
                }
            }
        }
        
        output[row * width + col] = sum;
    }
}
"""

def gpu_convolution(image, conv_kernel):
    height, width = image.shape
    kernel_size = conv_kernel.shape[0]
    
    # Flatten arrays
    image_flat = image.flatten().astype(np.float32)
    kernel_flat = conv_kernel.flatten().astype(np.float32)
    output_flat = np.zeros_like(image_flat)
    
    # Compile kernel
    program = Program(source=convolution_kernel, name="convolution_2d")
    kernel = program.compile().get_function("convolution_2d")
    
    # Allocate device memory
    d_input = core.cuMemAlloc(image_flat.nbytes)
    d_output = core.cuMemAlloc(output_flat.nbytes)
    d_kernel = core.cuMemAlloc(kernel_flat.nbytes)
    
    # Copy data
    core.cuMemcpyHtoD(d_input, image_flat)
    core.cuMemcpyHtoD(d_kernel, kernel_flat)
    
    # Launch kernel
    block_size = 16
    grid_x = (width + block_size - 1) // block_size
    grid_y = (height + block_size - 1) // block_size
    
    kernel.launch(
        grid=(grid_x, grid_y),
        block=(block_size, block_size),
        args=(d_input, d_output, d_kernel, 
              np.int32(width), np.int32(height), np.int32(kernel_size))
    )
    
    # Copy result back
    core.cuMemcpyDtoH(output_flat, d_output)
    
    # Cleanup
    core.cuMemFree(d_input)
    core.cuMemFree(d_output)
    core.cuMemFree(d_kernel)
    
    return output_flat.reshape(height, width)

# Test convolution
image = np.random.random((100, 100)).astype(np.float32)
edge_kernel = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]], dtype=np.float32)
result = gpu_convolution(image, edge_kernel)
print(f"Convolution completed. Output shape: {result.shape}")

## 10. Lab
### Exercise 1: Vector Operations
Write a CUDA kernel that performs element-wise multiplication of two vectors.

In [ ]:
# Your solution here
multiply_kernel_source = """
// TODO: Implement vector multiplication kernel
"""

def vector_multiply(a, b):
    # TODO: Implement the wrapper function
    pass

### Exercise 2: Reduction Operation
Implement a parallel reduction to find the maximum value in an array.

In [ ]:
# Your solution here
max_reduction_source = """
// TODO: Implement reduction kernel
"""

def find_max_gpu(arr):
    # TODO: Implement the wrapper function
    pass

### Exercise 3: Matrix Transpose
Write a kernel that transposes a matrix efficiently using shared memory.

In [ ]:
# Your solution here
transpose_kernel_source = """
// TODO: Implement matrix transpose kernel with shared memory
"""

def matrix_transpose_gpu(matrix):
    # TODO: Implement the wrapper function
    pass

### Exercise 4: Performance Comparison
Compare the performance of your GPU implementations with their CPU counterparts.

In [ ]:
def benchmark_operations():
    # TODO: Implement benchmarking code
    pass

## Solutions to Exercises
## Solution 1: Vector Multiplication

In [ ]:
multiply_kernel_source = """
extern "C" __global__ void vector_multiply(float *a, float *b, float *c, int n) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < n) {
        c[i] = a[i] * b[i];
    }
}
"""

def vector_multiply(a, b):
    assert len(a) == len(b), "Vectors must have same length"
    n = len(a)
    
    # Compile kernel
    program = Program(source=multiply_kernel_source, name="vector_multiply")
    kernel = program.compile().get_function("vector_multiply")
    
    # Allocate device memory
    d_a = core.cuMemAlloc(a.nbytes)
    d_b = core.cuMemAlloc(b.nbytes)
    d_c = core.cuMemAlloc(a.nbytes)
    
    # Copy data
    core.cuMemcpyHtoD(d_a, a)
    core.cuMemcpyHtoD(d_b, b)
    
    # Launch kernel
    block_size = 256
    grid_size = (n + block_size - 1) // block_size
    
    kernel.launch(
        grid=(grid_size,),
        block=(block_size,),
        args=(d_a, d_b, d_c, np.int32(n))
    )
    
    # Get result
    c = np.zeros_like(a)
    core.cuMemcpyDtoH(c, d_c)
    
    # Cleanup
    core.cuMemFree(d_a)
    core.cuMemFree(d_b)
    core.cuMemFree(d_c)
    
    return c

# Test
a = np.arange(1000, dtype=np.float32)
b = np.arange(1000, dtype=np.float32)
result = vector_multiply(a, b)
expected = a * b
print(f"Vector multiplication correct: {np.allclose(result, expected)}")

### Best Practices Summary

1. **Always initialize CUDA properly** with cuInit(0)
2. **Manage memory carefully** - allocate, copy, free in proper order
3. **Handle errors gracefully** - wrap CUDA calls in try-catch blocks
4. **Use appropriate block sizes** - typically 128, 256, or 512 threads
5. **Consider memory access patterns** - coalesced access is faster
6. **Use shared memory** for data reuse within blocks
7. **Profile your code** - use events for timing
8. **Clean up resources** - always free memory and destroy contexts

This tutorial provides a solid foundation for using cuda.core effectively. Remember that low-level CUDA programming requires careful attention to detail, but it offers maximum performance and flexibility for GPU computing tasks.

## Resources
CUDA Python Reference: https://numba.pydata.org/numba-doc/dev/cuda-reference/

Repository: https://github.com/NVIDIA/cuda-python 